In [ ]:
import csv
from pathlib import Path
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
home = Path.home()
fields = home/'group'/'project'/'results'/'hfss'/'220414_cres_antenna_asbuilt_fields'

dipole_fields = home/'group'/'project'/'results'/'hfss'/'220302_cres_antenna_fields'/'220302_cres_dipole_rE_phi_sweep.csv'
synca_fields = home/'group'/'project'/'results'/'hfss'/'220414_cres_antenna_asbuilt_fields'/'220414_cres_antenna_asbuilt_fields_phi.csv'

def ParseCSV(file):
    
    df = pd.read_csv(file)
    #with open(file, 'r') as infile:
    #for i, line in enumerate(csv.reader(infile)):
    #       print(line)
    
    return df


In [ ]:
files = list(fields.iterdir())

In [ ]:
for i in range(len(files)):
    print(files[i])

In [ ]:
data_list = []


for i in range(len(files)):
    
    if files[i].name.endswith('.csv'):

        frame = ParseCSV(files[i])
    
        data_list.append(frame)


In [ ]:
data_list[0]

# plot total fields

In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1,1,1, projection='polar')

plot_keys = ['mag(rETotal) [V]', 'mag(rEPhi) [V]', 'mag(rETheta) [V]']
labels = [r'$|\vec E|$', r'$E_\phi$', r'$E_\theta$']
for n, index in enumerate([1]):

    phi_sweep = data_list[index]

    thetas = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().index.array)
    
    for m, key in enumerate(plot_keys):
        
        re_comp = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = key).mean().array)
        if m == 0:
            norm = np.max(re_comp)

        ax.plot(np.radians(np.arange(90, -92, -2)), re_comp / norm, label=labels[m])

ax.set_thetamin(-90)
ax.set_thetamax(90)
#ax.set_theta_zero_location('N')
#ax.set_theta_direction(-1)

#ax.set_rmin(0)
#ax.set_rmax(11)

ax.legend(loc=(0.66,0.82))
ax.set_title('Normalized Electric Field Magnitude (AU)')

xticks = ax.set_xticks(np.radians(np.arange(90, -120, -30)))
xticklabels = ax.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])
#ax.set_xlabel(r'Zenith Angle (deg)', loc='center', y=100 )
#ax.text(np.radians(90), 1,'Zenith Angle (deg)')
#ax.text(np.radians(-55), 1, 'Electric Field (mV/m)', rotation='vertical')

plt.tight_layout()
name = '220720_synca_hfss_elevation.png'
save = home/'group'/'project'/'plots'/'hfss'/'synca'
save.mkdir(parents=True, exist_ok=True)

#plt.savefig(save/name, dpi=128)
#####


# Comparison

In [ ]:
lmc_data = home/'group'/'project'/'results'/'hfss'/'example.txt'
temp = []

with open(lmc_data, 'r') as infile:
    reader = csv.reader(infile, delimiter=' ')
    for i, line in enumerate(reader):
        temp.append(np.array(line, dtype=np.float64))
        if i % 500000 == 499999:
            print(i+1)
    print('Done!')
    
    
lmc_data = pd.DataFrame(
    np.array(temp),
    columns = ['theta', 'time', 'x', 'y', 'z', 'Ex', 'Ey', 'Ez']
)


lmc_phi_pol = []
lmc_theta_pol = []
lmc_total = []

for angle in np.arange(0, 180, 1):
    
    #mag.append(np.mean(np.sqrt(data[data['theta']==angle]['Ex']**2+data[data['theta']==angle]['Ey']**2+data[data['theta']==angle]['Ez']**2)))
    lmc_phi_pol.append(np.mean(abs(lmc_data[lmc_data['theta']==angle]['Ey'])**1))
    lmc_theta_pol.append(np.mean(abs(np.cos(np.radians(angle)) * lmc_data[lmc_data['theta']==angle]['Ex'] - np.sin(np.radians(angle)) * lmc_data[lmc_data['theta']==angle]['Ez'])**1))
    
    lmc_total.append(np.mean(abs(np.sqrt(lmc_data[lmc_data['theta']==angle]['Ex']**2 + lmc_data[lmc_data['theta']==angle]['Ey']**2+lmc_data[lmc_data['theta']==angle]['Ez']**2))))

##
lmc_norm = np.max(lmc_total)
lmc_total = np.array(lmc_total)/lmc_norm
##
lmc_norm_phi = np.max(lmc_phi_pol)
lmc_phi_pol = np.array(lmc_phi_pol)/lmc_norm_phi
##
lmc_norm_theta = np.max(lmc_theta_pol)
lmc_theta_pol = np.array(lmc_theta_pol)/lmc_norm_theta


In [ ]:
data_list = []


for file in [dipole_fields, synca_fields]:
    
    frame = ParseCSV(file)
    data_list.append(frame)


In [ ]:
data_list[0]

In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
fig = plt.figure(figsize=(23, 9))
ax1 = fig.add_subplot(1,3,2, projection='polar')

plot_keys = ['mag(rEPhi) [mV]', 'mag(rEPhi) [V]',]
labels = ['Crossed-dipole', 'PCB Crossed-dipole', ]
for n, index in enumerate([0, 1]):

    phi_sweep = data_list[index]

    thetas = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().index.array)
    
    for m, key in enumerate([0]):
        
        re_comp = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().array)
        if m == 0:
            norm = np.max(re_comp)

        ax1.plot(np.radians(np.arange(90, -92, -2)), 10*np.log10(re_comp / norm), label=labels[n], lw=3)
        
ax1.plot(np.radians(np.arange(90, -90, -1)), 10*np.log10(lmc_phi_pol), color='k', linestyle=':', label='Simulated Electron', lw=4)

ax1.set_thetamin(-90)
ax1.set_thetamax(90)
ax1.set_rmin(-10)
ax1.set_rmax(1.)
xticks = ax1.set_xticks(np.radians(np.arange(90, -120, -30)))
xticklabels = ax1.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])

ax1.text(np.radians(-130), -1.5, 'dB', rotation=0)
ax1.text(np.radians(0), -6 ,r'$\phi$ = 0$^\circ$')
ax1.text(np.radians(-95), 4.5, '(b)', fontsize=32)
ax1.set_title(r'$|E_{\phi}|$-normalized, $\theta$-sweep')
#ax.legend(loc=(0.66,0.82))
#ax.set_title(r'Normalized $E_\phi$ Magnitude (dB)')

#####################

ax2 = fig.add_subplot(1,3,3, projection='polar')

plot_keys = ['mag(rETotal) [mV]', 'mag(rETotal) [V]',]
labels = ['Crossed-dipole', 'PCB Crossed-dipole', ]
for n, index in enumerate([0, 1]):

    phi_sweep = data_list[index]

    thetas = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().index.array)
    
    for m, key in enumerate([0]):
        
        re_comp = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().array)
        if m == 0:
            norm = np.max(re_comp)

        ax2.plot(np.radians(np.arange(90, -92, -2)), 10*np.log10(re_comp / norm), lw=3)
        
        diff = 10*np.log10(re_comp / norm)[0:-1] - 10*np.log10(lmc_theta_pol[np.arange(0, 180, 2)])
        #print(np.arange(90, -92,-2)[0:-1])
        #print(np.arange(90, -90, -1)[np.arange(0, 180, 2)])
        
        #print(diff)
        print(np.flip(np.sort(abs(diff)))[0:14])
        
        print(
            10*np.log10(re_comp / norm)[0:-1][np.flip(np.argsort(abs(diff)))][0:14], 
            10*np.log10(lmc_theta_pol[np.arange(0, 180, 2)][np.flip(np.argsort(abs(diff)))])[0:14]
        )
        print(
            np.arange(90, -92, -2)[0:-1][np.flip(np.argsort(abs(diff)))][0:30], 
            #np.arange(90, -90, -1)[np.arange(0, 180, 2)][np.flip(np.argsort(abs(diff)))][0:14]
        )


#ax2.plot(np.radians(np.arange(90, -90, -1)), 10*np.log10(lmc_theta_pol), color='k', linestyle=':', lw=4)
ax2.plot(np.radians(np.arange(90, -90, -1)), 10*np.log10(lmc_total), color='k', linestyle=':', lw=4)

ax2.set_thetamin(-90)
ax2.set_thetamax(90)
ax2.set_rmin(-6)
ax2.set_rmax(1.)
xticks = ax2.set_xticks(np.radians(np.arange(90, -120, -30)))
xticklabels = ax2.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])

ax2.text(np.radians(-75), -10 ,r'$\phi$ = 0$^\circ$')
ax2.text(np.radians(-130), -4, 'dB', rotation=0)
ax2.text(np.radians(-95), 8.75, '(c)', fontsize=32)
ax2.set_title(r'$|E_{\theta}|$-normalized, $\theta$-sweep')

########################

ax3 = fig.add_subplot(1,3,1, projection='polar')

plot_keys0 = ['mag(rETotal) [mV]']
plot_keys1 = ['mag(rETotal) [V]']
plot_keys = [plot_keys0, plot_keys1]

labels = ['Crossed-dipole', 'PCB Crossed-dipole']
for n, index in enumerate([0, 1]):

    theta_sweep = data_list[index]

    thetas = np.array(theta_sweep.pivot(index='Theta [deg]', columns='Phi [deg]', values = plot_keys[n][0]).mean().index.array)
    
    for m, key in enumerate(plot_keys[n]):
        
        re_comp = np.array(theta_sweep.pivot(index='Theta [deg]', columns='Phi [deg]', values = key).mean().array)
        if m == 0:
            norm = np.max(re_comp)

        ax3.plot(np.radians(np.arange(0, 362, 2)), 10 * np.log10(re_comp / norm), lw=3,)

ax3.plot(np.radians(np.arange(0, 360, 1)), np.zeros(360),color='k', linestyle=':', lw=4)
#ax.set_thetamin(-90)
#ax.set_thetamax(90)
#ax.set_theta_zero_location('N')
#ax.set_theta_direction(-1)

ax3.set_rmin(-6)
ax3.set_rmax(1)

#ax.legend(loc=(0.26,0.2))
#ax.set_title('Normalized Electric Field Magnitude ')

#xticks = ax.set_xticks(np.radians(np.arange(90, -120, -30)))
#xticklabels = ax.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])
#ax.set_xlabel(r'Zenith Angle (deg)', loc='center', y=100 )
ax3.text(np.radians(-110), -2 ,r'$\theta$ = 90$^\circ$')
ax3.text(np.radians(40), -3, 'dB', rotation=30)
ax3.text(np.radians(-95), 3.2, '(a)', fontsize=32)
ax3.set_title(r'$|\vec E|$-normalized, $\phi$-sweep')

###############################

fig.legend(loc=8, ncol=3)

#plt.tight_layout()
name = '220916_field_mag_comparison.png'
save = home/'group'/'project'/'plots'/'hfss'/'synca'
save.mkdir(parents=True, exist_ok=True)
#plt.savefig(save/name, dpi=128)




In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
clist = sns.color_palette('deep', n_colors=10)
fig = plt.figure(figsize=(23, 9))

ax1 = fig.add_subplot(1,3,1, projection='polar')
#plot_keys0 = ['mag(rETotal) [mV]']
#plot_keys1 = ['mag(rETotal) [V]']
#plot_keys = [plot_keys0, plot_keys1]
plot_keys = [['mag(rETotal) [mV]'], ['mag(rETotal) [V]'],]

labels = ['Crossed-dipole', 'PCB Crossed-dipole']
norms = []
for n, index in enumerate([0, 1]):

    phi_sweep = data_list[index]

    thetas = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n][0]).mean().index.array)
    
    for m, key in enumerate(plot_keys[n]):
        
        #re_comp = np.array(theta_sweep.pivot(index='Theta [deg]', columns='Phi [deg]', values = key).mean().array)
        re_comp = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n][0]).mean().array)
        if m == 0:
            norm = np.max(re_comp)

        ax1.plot(np.radians(np.arange(90, -92, -2)), 10 * np.log10(re_comp / norm), lw=3, color=clist[n])
        
        diff = 10*np.log10(re_comp / norm)[0:-1] - 10*np.log10(lmc_total)[np.arange(0, 180, 2)]
        print(np.flip(np.sort(abs(diff)))[0:40])
        print(np.arange(0, 180, 2)[np.flip(np.argsort(abs(diff)))][0:40])

#ax3.plot(np.radians(np.arange(0, 360, 1)), np.zeros(360),color='k', linestyle=':', lw=4)
ax1.plot(np.radians(np.arange(90, -90, -1)), 10*np.log10(lmc_total), color='k', linestyle=':', lw=4)
ax1.set_thetamin(-90)
ax1.set_thetamax(90)
ax1.set_rmin(-10)
ax1.set_rmax(1)
xticks = ax1.set_xticks(np.radians(np.arange(90, -120, -30)))
xticklabels = ax1.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])

yticks = ax1.set_yticks(np.arange(-10, 2, 2))
yticklabels = ax1.set_yticklabels(np.arange(-10,2,2))
#ax3.set_theta_zero_location('N')
#ax3.set_theta_direction(-1)


#ax.legend(loc=(0.26,0.2))
#ax.set_title('Normalized Electric Field Magnitude ')

#xticks = ax.set_xticks(np.radians(np.arange(90, -120, -30)))
#xticklabels = ax.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])
#ax.set_xlabel(r'Zenith Angle (deg)', loc='center', y=100 )
ax1.text(np.radians(0), -7 ,r'$\phi$ = 0$^\circ$')
ax1.text(np.radians(-125), -3.5, 'dB', rotation=0)
ax1.text(np.radians(-95), 4.5, '(a)', fontsize=32)
ax1.set_title(r'$|\vec E|$-normalized, $\theta$-sweep')

############
ax2 = fig.add_subplot(1,3,2, projection='polar')

plot_keys = ['mag(rEPhi) [mV]', 'mag(rEPhi) [V]',]
labels = ['Crossed-dipole', 'PCB Crossed-dipole', ]
for n, index in enumerate([0, 1]):

    phi_sweep = data_list[index]

    thetas = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().index.array)
    
    for m, key in enumerate([0]):
        
        re_comp = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().array)
        if m == 0:
            norm = np.max(re_comp)

        ax2.plot(np.radians(np.arange(90, -92, -2)), 10*np.log10(re_comp / norm), label=labels[n], lw=3)
    
        
        diff = 10*np.log10(re_comp / norm)[0:-1] - 10*np.log10(lmc_phi_pol)[np.arange(0, 180, 2)]
        print(np.flip(np.sort(abs(diff)))[0:40])
        print(np.arange(0, 180, 2)[np.flip(np.argsort(abs(diff)))][0:40])
        
        #print(np.flip(np.sort(abs(diff)))[0:12])
        #print(np.arange(90, -90, -2)[np.flip(np.argsort(abs(diff)))][0:12])
        
ax2.plot(np.radians(np.arange(-90, 90, 1)), 10*np.log10(lmc_phi_pol), color='k', linestyle=':', label='Simulated Electron', lw=4)

ax2.set_thetamin(-90)
ax2.set_thetamax(90)
ax2.set_rmin(-10)
ax2.set_rmax(1.)
xticks = ax2.set_xticks(np.radians(np.arange(90, -120, -30)))
xticklabels = ax2.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])

yticks = ax2.set_yticks(np.arange(-10, 2, 2))
yticklabels = ax2.set_yticklabels(np.arange(-10,2,2))

ax2.text(np.radians(-125), -3.5, 'dB', rotation=0)
ax2.text(np.radians(0), -7 ,r'$\phi$ = 0$^\circ$')
ax2.text(np.radians(-95), 4.5, '(b)', fontsize=32)
ax2.set_title(r'$|E_{\phi}|$-normalized, $\theta$-sweep')
#ax.legend(loc=(0.66,0.82))
#ax.set_title(r'Normalized $E_\phi$ Magnitude (dB)')

#####################

ax3 = fig.add_subplot(1,3,3, projection='polar')

plot_keys = ['mag(rETheta) [mV]', 'mag(rETheta) [V]',]
labels = ['Crossed-dipole', 'PCB Crossed-dipole', ]
for n, index in enumerate([0, 1]):

    phi_sweep = data_list[index]

    thetas = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().index.array)
    
    for m, key in enumerate([0]):
        
        re_comp = np.array(phi_sweep.pivot(index='Phi [deg]', columns='Theta [deg]', values = plot_keys[n]).mean().array)
        if m == 0:
            norm = np.max(re_comp)

        ax3.plot(np.radians(np.arange(90, -92, -2)), 10*np.log10(re_comp / norm), lw=3)
        
        diff = 10*np.log10(re_comp / norm)[0:-1] - 10*np.log10(lmc_theta_pol)[np.arange(0, 180, 2)]
        print(np.flip(np.sort(abs(diff)))[0:40])
        print(np.arange(0, 180, 2)[np.flip(np.argsort(abs(diff)))][0:40])
        
        #print(np.arange(90, -92,-2)[0:-1])
        #print(np.arange(90, -90, -1)[np.arange(0, 180, 2)])
        
        #print(diff)
        #print(np.flip(np.sort(abs(diff)))[0:14])
        
        #print(
        #    10*np.log10(re_comp / norm)[0:-1][np.flip(np.argsort(abs(diff)))][0:14], 
        #    10*np.log10(lmc_theta_pol[np.arange(0, 180, 2)][np.flip(np.argsort(abs(diff)))])[0:14]
        #)
        #print(
        #    np.arange(90, -92, -2)[0:-1][np.flip(np.argsort(abs(diff)))][0:30], 
            #np.arange(90, -90, -1)[np.arange(0, 180, 2)][np.flip(np.argsort(abs(diff)))][0:14]
        #)


ax3.plot(np.radians(np.arange(90, -90, -1)), 10*np.log10(lmc_theta_pol), color='k', linestyle=':', lw=4)
#ax2.plot(np.radians(np.arange(90, -90, -1)), 10*np.log10(lmc_total), color='k', linestyle=':', lw=4)

ax3.set_thetamin(-90)
ax3.set_thetamax(90)
ax3.set_rmin(-10)
ax3.set_rmax(1.)

yticks = ax3.set_yticks(np.arange(-10,2, 2))
yticklabels = ax3.set_yticklabels(np.arange(-10,2,2))

xticks = ax3.set_xticks(np.radians(np.arange(90, -120, -30)))
xticklabels = ax3.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])

ax3.text(np.radians(-75), -5 ,r'$\phi$ = 0$^\circ$')
ax3.text(np.radians(-125), -3.5, 'dB', rotation=0)
ax3.text(np.radians(-95), 4.5, '(c)', fontsize=32)
ax3.set_title(r'$|E_{\theta}|$-normalized, $\theta$-sweep')

########################

###############################

fig.legend(loc=8, ncol=3)

#plt.tight_layout()
name = '221101_field_mag_comparison_theta_only.png'
save = home/'group'/'project'/'plots'/'hfss'/'synca'
save.mkdir(parents=True, exist_ok=True)
#plt.savefig(save/name, dpi=128)




In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
fig = plt.figure(figsize=(12, 9))

########################

ax1 = fig.add_subplot(1,1,1, projection='polar')

plot_keys = [['mag(rETotal) [mV]'], ['mag(rETotal) [V]']]

labels = ['Crossed-dipole', 'PCB Crossed-dipole']
for n, index in enumerate([0, 1]):

    theta_sweep = data_list[index]

    thetas = np.array(theta_sweep.pivot(index='Theta [deg]', columns='Phi [deg]', values = plot_keys[n][0]).mean().index.array)
    
    for m, key in enumerate(plot_keys[n]):
        
        re_comp = np.array(theta_sweep.pivot(index='Theta [deg]', columns='Phi [deg]', values = key).mean().array)
        if m == 0:
            norm = np.max(re_comp)
            
        diff = 10 * np.log10(re_comp / norm) - np.zeros(181)
        print(np.flip(np.sort(abs(diff)))[0:40])
        print(np.arange(0, 362, 2)[np.flip(np.argsort(abs(diff)))][0:40])

        ax1.plot(np.radians(np.arange(0, 362, 2)), 10 * np.log10(re_comp / norm), lw=3, label=labels[n])

ax1.plot(np.radians(np.arange(0, 360, 1)), np.zeros(360),color='k', linestyle=':', lw=4, label='Simulated Electron')
#ax.set_thetamin(-90)
#ax.set_thetamax(90)
#ax.set_theta_zero_location('N')
#ax.set_theta_direction(-1)

ax1.set_rmin(-1)
ax1.set_rmax(0.25)

#ax.legend(loc=(0.26,0.2))
#ax.set_title('Normalized Electric Field Magnitude ')

#xticks = ax.set_xticks(np.radians(np.arange(90, -120, -30)))
#xticklabels = ax.set_xticklabels(labels=[fr'{n}$^\circ$' for n in np.arange(0, 210, 30)])
#ax.set_xlabel(r'Zenith Angle (deg)', loc='center', y=100 )
ax1.text(np.radians(-110), -2 ,r'$\theta$ = 90$^\circ$')
ax1.text(np.radians(23), 0.37, 'dB', rotation=30)
ax1.text(np.radians(-95), 3.2, '(a)', fontsize=32)
ax1.set_title(r'$|\vec E|$-normalized, $\phi$-sweep')

###############################

fig.legend(loc=1, ncol=1)

#plt.tight_layout()
name = '221109_field_mag_comparison_phi_only.png'
save = home/'group'/'project'/'plots'/'hfss'/'synca'
save.mkdir(parents=True, exist_ok=True)
plt.savefig(save/name, dpi=128)


